<a href="https://colab.research.google.com/github/rb58853/images_RIS-ML-Conv-NLP/blob/main/sam/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### install dependences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

### tool: create image from a input mask

In [ ]:
from PIL import Image

def mask_image(mask, raw_image):
    weigth, heigth = raw_image.size
    new_image = Image.new('RGBA', (weigth, heigth), (0, 0, 0, 0))

    original_pixles = raw_image.load()
    pixels = new_image.load()

    # print("\n\n------------------------NEW MASK------------------------------\n" + str(mask))

    for i in range (heigth):
        for j in range (weigth):
            if mask[i,j]:
                pixels[j, i] = original_pixles[j,i]
            else:
                pass
    return new_image

### Working with SAM

##### import dependences and model from hugginface

In [ ]:
import torch
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

##### select and import image by process

In [ ]:
img_url = "/content/drive/MyDrive/Tesis/Clip_tests/3.jpg"

raw_image = Image.open(img_url).convert("RGB")
input_points = [[[450, 600]]]  # 2D location of a window in the image

##### masks from image

In [ ]:
inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to(device)
outputs = model(**inputs)

#Estas son mascaras booleanas True implica que el pixel en cuestion interesa, y False implica que no es de interes, es Fondo de esa capa
masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu()
)

In [ ]:
for m1 in masks:
  for m2 in m1:
    print("La cantidad de mascaras es: "+str(len(m2)))
    for i, m3 in zip(range(len(m2)),m2):
      new_image = mask_image(m3, raw_image)
      new_image.save(f'/content/drive/MyDrive/Tesis/sam_images/{i}.png', 'PNG')



La cantidad de mascaras es: 3
